In [1]:
####################################################################################################
# Import libraries

from pathlib import Path
import folium
import shapefile
import json
from xlrd import open_workbook
from openpyxl import load_workbook
import csv
from redline_objects import *

###############################################################################
# Locate project files

filename = input("Input excel file name with the correct extension >> ")
redline_xl = Path('Input').joinpath(filename)
wmg30_trails_shapefile = "Input/wmg30_trails_shapefile/wmg30_trails_shapefile.shp"
amc_4000_peaks = "Input/amc_4k_peaks.csv"
tabs_expected = json.load(open("Input/tabs_expected.json"))

###############################################################################
# Read geometries from the shapefile
trails_dict = dict()    # keys are trail_ID and values are trail objects with 
alt_ids = dict()    # keys are alt_ID and values are trail_ID

# Read shapefile
reader = shapefile.Reader(wmg30_trails_shapefile)
to_json = []
for sr in reader.shapeRecords():
    trail = Trail(sr.record['Trail_ID'])
    trail.geom = [(lat, lng) for (lng, lat) in sr.shape.__geo_interface__["coordinates"]]
    if len(sr.record['Alt_ID']) > 0:
        trail.alt_ID = sr.record['Alt_ID']
        alt_ids[sr.record['Alt_ID']] = sr.record['Trail_ID']
    trails_dict[sr.record['Trail_ID']] = trail

###############################################################################
# Scan spreadsheet for trails and update attributes
trails_in_shapefile = list(trails_dict.keys())
trails_without_geom = []  # trails found in spreadsheet that are not in shapefile
trails_to_draw = []  # trails in both spreadsheet and shapefile

redline_workbook = Spreadsheet(redline_xl, tabs_expected)
for tab in redline_workbook.tab_names:
    tab_object = TabObject(redline_workbook, tab)
    trail_attr_dict = tab_object.trail_attr_dict
    for k in trail_attr_dict.keys():
        if k in trails_in_shapefile:
            trails_dict[k].mileage = trail_attr_dict[k][0]
            trails_dict[k].miles_todo = trail_attr_dict[k][1]
            trails_to_draw.append(k)
        elif k in alt_ids.keys():
            trails_dict[alt_ids[k]].mileage = trail_attr_dict[k][0]
            trails_dict[alt_ids[k]].miles_todo = trail_attr_dict[k][1]
            trails_to_draw.append(alt_ids[k])
        else:
            trails_without_geom.append(k)

###############################################################################
# Draw trails onto a folium map
m = folium.Map(location=[44.1, -71.4], tiles='Stamen Terrain', zoom_start=10,
               min_zoom=4, control_scale=True, max_bounds=True)

# Add trails to the map
trails_layer = folium.FeatureGroup(name='Trails')

for tr in trails_to_draw:
    trail = trails_dict[tr]
    if trail.miles_todo == 0:
        trail_color = 'red'
    else:
         trail_color = 'blue' 
    popup_text = "<strong> Trail Name: </strong>{0}<br>" \
                     "<strong> WMG Section: </strong>{1}<br>" \
                     "<strong> Spreadsheet Tab: </strong>{2}<br>" \
                     "<strong> Mileage Total: </strong>{3}<br>" \
                     "<strong> Mileage To Do: </strong>{4}" \
            .format(trail.trail_name, tabs_expected[trail.tab],
                    trail.tab, trail.mileage, trail.miles_todo)
    popup = folium.Popup(popup_text, max_width=600)
    folium.vector_layers.PolyLine(trail.geom, popup, tooltip=trail.trail_name,
                                  color=trail_color, weight=1).add_to(trails_layer)
trails_layer.add_to(m)

# Add 4000 footers to the map
peaks_layer = folium.FeatureGroup(name='4000 Footers')
with open(amc_4000_peaks, 'r') as csvfile:
    peaks_4k = csv.DictReader(csvfile)
    for row in peaks_4k:
        peak_icon = folium.features.CustomIcon(icon_image='Input/Icons/triangle.png', icon_size=(16,16))
        folium.Marker((row['LAT'], row['LNG']), tooltip="{0} ({1} ft)".format(row['Peak'], row['Elevation']),
                      icon=peak_icon).add_to(peaks_layer)
peaks_layer.add_to(m)
        
folium.LayerControl(autoZIndex=False, collapsed=False).add_to(m)

if len(trails_without_geom) > 0:
    print("The geometry of the following trails is unavailable and could not be drawn:<br>")
    print("    (Tab, Trail Name)")
    i = 1
    for value in trails_without_geom:
        print("{0}.    {1}".format(i, value))
        i += 1
        
# Path('Output').mkdir(mode=511)
m.save("Output/redline_map_output.html")
print("\n Map saved to the file 'Output/redline_map_output.html'")

The geometry of the following trails is unavailable and could not be drawn:<br>
    (Tab, Trail Name)
1.    Cannon, Veterans Trail
2.    Cannon, Dilly Trail
3.    Waterville, Flume Brook Trail
4.    Waterville, Scaur Trail
5.    Waterville, Red Hill Loop Trail
6.    Carters, Middle Mountain Connector
7.    NorthernNH, Boothman Spring Spur Trail
8.    NorthernNH, Magalloway River Trail

 Map saved to the file 'Output/redline_map_output.html'
